In [1]:
import tushare as ts
import pandas as pd
df = ts.pro_bar(ts_code='688256.SH', start_date='19910101', end_date='20220923',adj='qfq',adjfactor=True)
stock_hist = df[['trade_date', 'open','close','high','low']]
stock_hist.rename(columns={'trade_date':'Date', 'open':'Open','high':'High','low':'Low','close':'Close'},inplace=True)
stock_hist = stock_hist[:5000]
stock_hist = stock_hist.iloc[::-1]
stock_hist['Date'] = pd.to_datetime(stock_hist['Date'])
stock_hist.set_index('Date', inplace=True)
# stock_hist = stock_hist.reset_index()
# stock_hist.drop(['index'],axis=1, inplace=True)
# PINGAN.dropna()
# PINGAN.interpolate(method='linear', limit_direction='forward', axis=0)
stock_hist.head()

/var/folders/wf/l689qmr574b9nlzwszjgltk00000gn/T/ipykernel_64423/3114993532.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_hist.rename(columns={'trade_date':'Date', 'open':'Open','high':'High','low':'Low','close':'Close'},inplace=True)


,Open,Close,High,Low
Date,,,,
2020-07-20,250.00,212.40,295.00,205.60
2020-07-21,208.00,274.00,285.60,202.00
2020-07-22,260.00,281.00,284.33,257.77
2020-07-23,274.92,281.50,297.77,268.00
2020-07-24,273.00,259.33,276.00,240.44


from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n1 = 10
    n2 = 20
    c = 0
    d = 0
    steps = {}

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        for k,v in self.steps.items():
            print(k + ':' + v)
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(stock_hist, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run(c=10,d=60,steps={'a':'cross', 'b':'above200'})
# bt.plot()

In [71]:
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats._trades

/Users/hanjie/Applications/venv2/lib/python3.8/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,3,127,132,166.43220,158.0156,-25.24980,-0.050571,2021-01-22,2021-01-29,7 days
1,59,122,132,159.31800,158.0156,-76.84160,-0.008175,2021-01-15,2021-01-29,14 days
2,61,150,158,153.10560,147.4148,-347.13880,-0.037169,2021-03-03,2021-03-15,12 days
3,76,208,232,117.93540,127.4890,726.07360,0.081007,2021-05-28,2021-07-02,35 days
4,113,315,334,86.06178,92.1432,687.20046,0.070663,2021-11-05,2021-12-02,27 days
5,107,357,360,97.19400,90.2076,-747.54480,-0.071881,2022-01-05,2022-01-10,5 days
6,6,395,401,86.13192,78.5000,-45.79152,-0.088607,2022-03-07,2022-03-15,8 days
7,120,387,401,80.59086,78.5000,-250.90320,-0.025944,2022-02-23,2022-03-15,20 days
8,9,451,460,55.43064,62.6532,65.00304,0.130299,2022-05-31,2022-06-14,14 days
9,171,444,460,54.83946,62.6532,1336.14954,0.142484,2022-05-20,2022-06-14,25 days


In [9]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy
import talib 

class xMACross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        ema1 = self.I(talib.EMA, self.data.Close, self.n1)
        ema2 = self.I(talib.EMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(ema1) > ema2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [4]:
from backtesting import Backtest, Strategy
from backtesting.test import SMA, GOOG
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy
import talib 

class MyBackTest(Backtest):
    def get_strategy(self):
        """Candlestick bar index of when the trade was entered."""
        return self._strategy
    
    strategy = property(get_strategy)

    #strategy, trades, equity, positions

def init(self):
    # In init() and in next() it is important to call the
    # super method to properly initialize the parent classes
    super(TranStrategy, self).init()
    
    # Precompute the two moving averages
    ema1 = self.I(talib.EMA, self.data.Close, self.n1)
    ema2 = self.I(talib.EMA, self.data.Close, self.n2)
    
    # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
    signal = (pd.Series(ema1) > ema2).astype(int).diff().fillna(0)
    signal = signal.replace(-1, 0)  # Upwards/long only
    
    # Use 95% of available liquidity (at the time) on each order.
    # (Leaving a value of 1. would instead buy a single share.)
    entry_size = signal * .95
            
    # Set order entry sizes using the method provided by 
    # `SignalStrategy`. See the docs.
    self.set_signal(entry_size=entry_size)
    
    # Set trailing stop-loss to 2x ATR using
    # the method provided by `TrailingStrategy`
    self.set_trailing_sl(2)

def next(self):
    pass
    
TranStrategy = type(
    'TranStrategy', 
    (SignalStrategy, TrailingStrategy),
    {
        'n1': 10,
        'n2': 25,
        'init': init,
        'next': next
    })

bt = MyBackTest(stock_hist, TranStrategy, commission=.002)

output = bt.run()
bt.strategy.trades
# bt.plot()
# output

In [47]:
def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

In [48]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70

    chains = {'buy':'aaa','sell':'dsd'}
    
    def init(self):
        # cccc
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [49]:
backtest = Backtest(stock_hist, System, commission=.002)
backtest.run()
backtest.plot()

Row(id='19649', ...)

In [50]:
from backtesting.test import GOOG

GOOG

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800


In [102]:
from abc import abstractmethod
from urllib.parse import parse_qs


class Person:
    @abstractmethod
    def init(self):
        '''
        print('this is super')
        '''
    
    def __init__(self, name):
        self._name = name

    def get_name(self):
        print('Getting name')
        return self._name

    def set_name(self, value):
        print('Setting name to ' + value)
        self._name = value

    def del_name(self):
        print('Deleting name')
        del self._name

    # Set property to use get_name, set_name
    # and del_name methods
    name = property(get_name, set_name, del_name, 'Name property')

class Kid(Person):
    param = ''
    def init(self, param):
        # super.init(self)
        self.param = param
        print('this is child - ' + self._name)

p = Person('Adam')
print(p.name)
p.name = 'John'
del p.name

k = Kid('Guagua')
k.init('Guagua')
# print(k.param)

Getting name
Adam
Setting name to John
Deleting name
this is child - Guagua
Guagua


In [86]:
def a():
    pass

a.__name__

'a'

In [103]:
from backtesting.test import SMA

from backtesting import Strategy
from backtesting.lib import crossover


class Sma4Cross(Strategy):
    n1 = 50
    n2 = 100
    n_enter = 20
    n_exit = 10
    
    def init(self):
        super.init()
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.sma_enter = self.I(SMA, self.data.Close, self.n_enter)
        self.sma_exit = self.I(SMA, self.data.Close, self.n_exit)
        
    def next(self):
        
        if not self.position:
            
            # On upwards trend, if price closes above
            # "entry" MA, go long
            
            # Here, even though the operands are arrays, this
            # works by implicitly comparing the two last values
            if self.sma1 > self.sma2:
                if crossover(self.data.Close, self.sma_enter):
                    self.buy()
                    
            # On downwards trend, if price closes below
            # "entry" MA, go short
            
            else:
                if crossover(self.sma_enter, self.data.Close):
                    self.sell()
        
        # But if we already hold a position and the price
        # closes back below (above) "exit" MA, close the position
        
        else:
            if (self.position.is_long and
                crossover(self.sma_exit, self.data.Close)
                or
                self.position.is_short and
                crossover(self.data.Close, self.sma_exit)):
                
                self.position.close()

In [104]:
from backtesting import Backtest
from backtesting.test import GOOG


backtest = Backtest(GOOG, Sma4Cross, commission=.002)

stats, heatmap = backtest.optimize(
    n1=range(10, 110, 10),
    n2=range(20, 210, 20),
    n_enter=range(15, 35, 5),
    n_exit=range(10, 25, 5),
    constraint=lambda p: p.n_exit < p.n_enter < p.n1 < p.n2,
    maximize='Equity Final [$]',
    max_tries=200,
    random_state=0,
    return_heatmap=True)

/Users/hanjie/Applications/venv2/lib/python3.8/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [105]:
heatmap

n1   n2   n_enter  n_exit
20   60   15       10        10102.86700
     80   15       10         9864.21924
     100  15       10        11003.21764
30   40   20       15        11771.28610
          25       15        16178.54842
                                ...     
100  200  15       10        13118.24766
          20       10        11308.46180
                   15        16350.94380
          25       10         8991.55294
          30       10         9953.07010
Name: Equity Final [$], Length: 177, dtype: float64

In [106]:
heatmap.sort_values().iloc[-3:]

n1   n2   n_enter  n_exit
100  120  15       10        18159.06414
     160  20       15        19216.54456
50   160  20       15        19565.69222
Name: Equity Final [$], dtype: float64